# OpenClassrooms - Ingenieur IA
# Projet 10 - Fly Me - Partie 2 : Modélisation avec LUIS
# Développez un chatbot pour réserver des vacances

## Objectif du projet : 
- **Construire un MVP qui aidera les employés de Fly Me à réserver facilement un billet d’avion pour leurs vacances**

## Plan :
- **Partie 1 : Analyse et préparation des données**
    - Chargement des données
    - Analyse des données brutes
    - Parsing des données pour LUIS
    - Sauvegarde des données parsées pour LUIS
    - Analyse des données parsées pour LUIS
    - Nettoyage des données parsées pour LUIS
    - Sauvegarde des données parsées et nettoyées pour LUIS
 
 
- **Partie 2 : Modélisation avec LUIS**
    - Chargement des données
    - Creation de l'application (LUISAuthoringClient)
    - Creation des exemples d'entrainement : Phrases (utterances), Entités (Entities) et Valeur des entités
        - Ajout d'exemples pour l'intention : OrderTravel
        - Ajout d'exemples pour l'intention : Greetings
        - Ajout d'exemples pour l'intention : None
    - Entrainement du modèle   
    - Publication du modèle
    - Evaluation du modèle
         - Creation du client de l'application (LUISRuntimeClient)
         - Evaluation sur le jeu de test
         
         
- **Partie 3 : Modélisation avec Microsoft Bot Framework**
    - Voir repository GitHub
    
    
- **Partie 4 : Tests**
    - Voir repository GitHub
    
    
- **Partie 5 : Déploiement**
    - Fait avec Azure + GitHub Actions


- **Partie 6 : Monitoring**
    - Fait avec Azure Application Insights + rédaction d'une méthodologie

## Script Partie 2 : Modélisation avec LUIS

### Remarque préliminaire :
- **Le code de ce script est découpé en fonctions, ce qui permet une meilleure lisibilité et une meilleure maintenabilité du code**

In [1]:
import json
import time
import uuid
import datetime
from pprint import pprint

import pandas as pd
import numpy as np

from azure.cognitiveservices.language.luis.authoring import LUISAuthoringClient
from azure.cognitiveservices.language.luis.authoring.models import ApplicationCreateObject
from azure.cognitiveservices.language.luis.runtime import LUISRuntimeClient

from msrest.authentication import CognitiveServicesCredentials

from sklearn.model_selection import train_test_split

# Chargement des données

In [2]:
df_data_luis = pd.read_csv('luis_parsed_cleaned_dataset.csv').replace({np.nan: ""})

In [3]:
df_data_luis

,text,intent,or_city,dst_city,str_date,end_date,budget
0,I'd like to book a trip to Atlantis from Capri...,book,Caprica,Atlantis,"Saturday, August 13, 2016",,1700
1,"Hello, I am looking to book a vacation from Go...",book,Gotham City,Mos Eisley,,,2100
2,Hello there i am looking to go on a vacation w...,book,,Gotham City,,,
3,"Hi I'd like to go to Caprica from Busan, betwe...",book,Busan,Caprica,"Sunday August 21, 2016","Wednesday August 31, 2016",
4,"Hello, I am looking to book a trip for 2 adult...",book,Kochi,Denver,,,"21,300"
...,...,...,...,...,...,...,...
1364,Hi I've got 9 days free and I'm looking for a ...,book,,,,,
1365,I need to get to Fortaleza on September 8th or...,book,,Fortaleza,September 8th,,
1366,We're finally going on vacation isn't that ama...,book,,,,,15600
1367,"Hi there, I'm looking for a place to get away ...",book,,,,,


## Données pour intention 'book'

In [4]:
df_data_luis_book_intent = df_data_luis[df_data_luis['intent']=='book']

In [5]:
df_data_luis_book_intent

,text,intent,or_city,dst_city,str_date,end_date,budget
0,I'd like to book a trip to Atlantis from Capri...,book,Caprica,Atlantis,"Saturday, August 13, 2016",,1700
1,"Hello, I am looking to book a vacation from Go...",book,Gotham City,Mos Eisley,,,2100
2,Hello there i am looking to go on a vacation w...,book,,Gotham City,,,
3,"Hi I'd like to go to Caprica from Busan, betwe...",book,Busan,Caprica,"Sunday August 21, 2016","Wednesday August 31, 2016",
4,"Hello, I am looking to book a trip for 2 adult...",book,Kochi,Denver,,,"21,300"
...,...,...,...,...,...,...,...
1364,Hi I've got 9 days free and I'm looking for a ...,book,,,,,
1365,I need to get to Fortaleza on September 8th or...,book,,Fortaleza,September 8th,,
1366,We're finally going on vacation isn't that ama...,book,,,,,15600
1367,"Hi there, I'm looking for a place to get away ...",book,,,,,


In [6]:
print(f"Taille du jeu de données : {len(df_data_luis_book_intent)}")

Taille du jeu de données : 1134


### Séparation des données

In [7]:
df_data_luis_book_intent_train, df_data_luis_book_intent_test = train_test_split(df_data_luis_book_intent, train_size=0.5, random_state=0)

In [8]:
df_data_luis_book_intent_train = df_data_luis_book_intent_train.reset_index(drop=True)

In [9]:
df_data_luis_book_intent_train

,text,intent,or_city,dst_city,str_date,end_date,budget
0,Any week long trips to Brasilia from Mannheim ...,book,Mannheim,Brasilia,august 30,september 15,
1,Hello! I recently won the internship lottery t...,book,Alexandria,Kobe,,,
2,Hi i want to go to burlington,book,,burlington,,,
3,hello!,book,,,,,
4,I want to leave Kabul on or after Sept 6 please,book,Kabul,,Sept 6,,
...,...,...,...,...,...,...,...
562,Need two tickets out of Buenos Aires!,book,Buenos Aires,,,,
563,YOU FLY OUT OF BEIJING?,book,BEIJING,,,,
564,"Hi there, I'd like to book a trip from Boston ...",book,Boston,,,,
565,Jerusalem to Belem 2 adults 4000 dollar budget...,book,Jerusalem,Belem,September third,,4000


In [10]:
print(f"Taille du jeu d'entrainement : {len(df_data_luis_book_intent_train)}")

Taille du jeu d'entrainement : 567


In [11]:
df_data_luis_book_intent_test = df_data_luis_book_intent_test.reset_index(drop=True)

In [12]:
df_data_luis_book_intent_test

,text,intent,or_city,dst_city,str_date,end_date,budget
0,Hey hey. Got 6 days off starting Sept 8. Flyin...,book,Cleveland,,Sept 8,,
1,"Hi yes, to Jerusalem please.",book,,Jerusalem,,,
2,You got any flights from New York to Naples de...,book,New York,Naples,Aug 27,Sep 15,
3,Hello. me and my grand child would like to go ...,book,,,,,
4,i need a trip under the radar from mannheim to...,book,mannheim,fukuoka,,august 30th,
...,...,...,...,...,...,...,...
562,Good afternoon. I'd like to book a business tr...,book,Mannheim,New York,,,
563,yes. i am going 2 bring my grand daughter with...,book,,,,,
564,This is a little short notice But I'm trying t...,book,San Diego,,Sept 8,17,
565,HI I NEED TO GO ON VACATION I LIVE IN WASHINGT...,book,,,,,


In [13]:
print(f"Taille du jeu de test : {len(df_data_luis_book_intent_test)}")

Taille du jeu de test : 567


## Données pour intention 'greeting'

In [14]:
df_data_luis_greeting_intent = df_data_luis[df_data_luis['intent']=='greeting']

In [15]:
df_data_luis_greeting_intent

,text,intent,or_city,dst_city,str_date,end_date,budget
40,Hi!,greeting,,,,,
43,Hi! I'd like to go to Boston from Mos Eisley o...,greeting,Mos Eisley,Boston,August 15th,,
48,Heyo!,greeting,,,,,
52,Good morning.,greeting,,,,,
63,Hello wozbot!,greeting,,,,,
...,...,...,...,...,...,...,...
1188,hi there. i really wanna pretend im somewhere ...,greeting,,,,,2900
1211,Guess what? I'm a recently married person look...,greeting,osaka,manaus,,,
1223,Hi,greeting,,,,,
1251,Hi,greeting,,,,,


In [16]:
print(f"Taille du jeu de données : {len(df_data_luis_greeting_intent)}")

Taille du jeu de données : 141


### Séparation des données

In [17]:
df_data_luis_greeting_intent_train, df_data_luis_greeting_intent_test = train_test_split(df_data_luis_greeting_intent, train_size=0.5, random_state=0)

In [18]:
df_data_luis_greeting_intent_train = df_data_luis_greeting_intent_train.reset_index(drop=True)

In [19]:
df_data_luis_greeting_intent_train

,text,intent,or_city,dst_city,str_date,end_date,budget
0,Hi there!,greeting,,,,,
1,hey,greeting,,,,,
2,"Hello, kind stranger.",greeting,,,,,
3,Hi there!,greeting,,,,,
4,Howdy! So how does this work? I'm not too good...,greeting,,,,,
...,...,...,...,...,...,...,...
65,hi my wife and i wanna bring our 7 little lads...,greeting,san diego,kochi,,,28500
66,Hi! Can you get me a trip from FOrtaleza to Pu...,greeting,FOrtaleza,Puebla,,,
67,EXCUSE ME,greeting,,,,,
68,You too!,greeting,,,,,


In [20]:
print(f"Taille du jeu d'entrainement : {len(df_data_luis_greeting_intent_train)}")

Taille du jeu d'entrainement : 70


In [21]:
df_data_luis_greeting_intent_test = df_data_luis_greeting_intent_test.reset_index(drop=True)

In [22]:
df_data_luis_greeting_intent_test

,text,intent,or_city,dst_city,str_date,end_date,budget
0,Good day. I live in Kabul and I want to take m...,greeting,Kabul,,,,
1,Hello there!,greeting,,,,,
2,hi there,greeting,,,,,
3,hey,greeting,,,,,
4,Heya,greeting,,,,,
...,...,...,...,...,...,...,...
66,Hi,greeting,,,,,
67,hello hello,greeting,,,,,
68,hiya,greeting,,,,,
69,Hi there,greeting,,,,,


In [23]:
print(f"Taille du jeu de test : {len(df_data_luis_greeting_intent_test)}")

Taille du jeu de test : 71


## Données pour intention 'none'

In [24]:
df_data_luis_none_intent = df_data_luis[df_data_luis['intent']=='none']

In [25]:
df_data_luis_none_intent = df_data_luis_none_intent.reset_index(drop=True)

In [26]:
print(f"Taille du jeu de données : {len(df_data_luis_none_intent)}")

Taille du jeu de données : 3


# Creation de l'application (LUISAuthoringClient)
- Fonction **create_luis_app** permettant la création de l'application LUIS

In [27]:
#config
authoring_key = '56dcb46dd5144af38db9bf6666aafed3'
authoring_endpoint = 'https://flymeluis-authoring.cognitiveservices.azure.com/'
prediction_key = '87488d948aba4eb39f8986e82d88d0a2'
prediction_endpoint = 'https://flyme-luis.cognitiveservices.azure.com/'

In [28]:
def create_luis_app():
    #we use a UUID to avoid name collisions
    app_name = "Fly Me App" + str(uuid.uuid4())
    version_id = "0.1"
    
    #create client
    client = LUISAuthoringClient(authoring_endpoint, CognitiveServicesCredentials(authoring_key))
    
    #define app basics
    app_definition = ApplicationCreateObject(name=app_name, initial_version_id=version_id, culture='en-us')

    #create app
    app_id = client.apps.add(app_definition)

    print("Created LUIS app with ID {}".format(app_id))
    
    return client, version_id, app_id

In [29]:
client, version_id, app_id = create_luis_app()

Created LUIS app with ID 52c42e40-1395-47e1-9fcb-d8e59de2d71b


# Creation des exemples d'entrainement : Phrases (Utterances), Entités (Entities) et Valeur des entités

In [30]:
def get_label(utterance, entity_name, value):
    utterance = utterance.lower()
    value = value.lower()
    return {
        'entity_name': entity_name,
        'start_char_index': utterance.find(value),
        'end_char_index': utterance.find(value) + len(value)
    }

## Ajout d'exemples pour l'intention : OrderTravel
- Fonction **add_order_travel_intent** permettant d'ajouter l'intention
- Fonction **add_order_travel_entities** permettant d'ajouter les entités
- Fonction **add_utterances_order_travel** permettant d'ajouter les utterances (les exemples)

In [31]:
intent_name_order_travel = "OrderTravelIntent"
def add_order_travel_intent(client, version_id, app_id):
    #add intent
    intent_id_order_travel = client.model.add_intent(app_id, version_id, intent_name_order_travel)

In [32]:
def add_order_travel_entities(client, version_id, app_id):
    #add Prebuilt entity
    client.model.add_prebuilt(app_id, version_id, prebuilt_extractor_names=["datetimeV2"])
    
    #add entities
    departure_city_name = "DepartureCity"
    departure_city_id = client.model.add_entity(app_id, version_id, name=departure_city_name)

    arrival_city_name = "ArrivalCity"
    arrival_city_id = client.model.add_entity(app_id, version_id, name=arrival_city_name)

    departure_date_name = "DepartureDate"
    departure_date_id = client.model.add_entity(app_id, version_id, name=departure_date_name)

    arrival_date_name = "ArrivalDate"
    arrival_date_id = client.model.add_entity(app_id, version_id, name=arrival_date_name)

    price_name = "Price"
    price_id = client.model.add_entity(app_id, version_id, name=price_name)

In [33]:
def add_utterances_order_travel(client, version_id, app_id, nb_utterances_to_add):

    utterances_order_travel = []

    for i in range(min(nb_utterances_to_add, len(df_data_luis_book_intent_train))):

        entity_labels = []

        if (df_data_luis_book_intent_train.iloc[i][2]!=''):
            entity_labels.append(get_label(df_data_luis_book_intent_train.iloc[i][0], "DepartureCity", df_data_luis_book_intent_train.iloc[i][2]))

        if (df_data_luis_book_intent_train.iloc[i][3]!=''):
            entity_labels.append(get_label(df_data_luis_book_intent_train.iloc[i][0], "ArrivalCity", df_data_luis_book_intent_train.iloc[i][3]))

        if (df_data_luis_book_intent_train.iloc[i][4]!=''):
            entity_labels.append(get_label(df_data_luis_book_intent_train.iloc[i][0], "DepartureDate", df_data_luis_book_intent_train.iloc[i][4]))

        if (df_data_luis_book_intent_train.iloc[i][5]!=''):
            entity_labels.append(get_label(df_data_luis_book_intent_train.iloc[i][0], "ArrivalDate", df_data_luis_book_intent_train.iloc[i][5]))

        if (df_data_luis_book_intent_train.iloc[i][6]!=''):
            entity_labels.append(get_label(df_data_luis_book_intent_train.iloc[i][0], "Price", df_data_luis_book_intent_train.iloc[i][6]))

        utterances_order_travel.append({
            'text': df_data_luis_book_intent_train.iloc[i][0],
            'intent_name': intent_name_order_travel,
            'entity_labels': entity_labels
        })
    
    print(f"Nb utterances_order_travel to add = {len(utterances_order_travel)}")
    
    #as we can only add 100 utterances per batch we use a loop to add all utterances
    j = len(utterances_order_travel)
    k = 0
    while ((j-100)>0):
        utterances_result = client.examples.batch(app_id, version_id, utterances_order_travel[k:k+100])
        j = j-100
        k = k+100

    utterances_result = client.examples.batch(app_id, version_id, utterances_order_travel[k:len(utterances_order_travel)])
    
    print("\nUtterances added to the {} intent".format(intent_name_order_travel))

In [34]:
#add intent
add_order_travel_intent(client, version_id, app_id)

#add entities
add_order_travel_entities(client, version_id, app_id)

#add utterances
add_utterances_order_travel(client, version_id, app_id, nb_utterances_to_add=len(df_data_luis_book_intent_train))

Nb utterances_order_travel to add = 567

Utterances added to the OrderTravelIntent intent


## Ajout d'exemples pour l'intention : Greetings
- Fonction **add_greetings_intent** permettant d'ajouter l'intention
- Fonction **add_utterances_greetings** permettant d'ajouter les utterances (les exemples)

In [35]:
intent_name_greetings = "GreetingsIntent"
def add_greetings_intent(client, version_id, app_id):
    #add intent
    intent_id_greetings = client.model.add_intent(app_id, version_id, intent_name_greetings)

In [36]:
def add_utterances_greetings(client, version_id, app_id, nb_utterances_to_add):

    utterances_greetings = []

    for i in range(min(nb_utterances_to_add, len(df_data_luis_greeting_intent_train), 100)):

        utterances_greetings.append({
            'text': df_data_luis_greeting_intent.iloc[i][0],
            'intent_name': intent_name_greetings,
        })   
        
    print(f"Nb utterances_greetings to add = {len(utterances_greetings)}")
    
    utterances_result = client.examples.batch(app_id, version_id, utterances_greetings)
    
    print("\nUtterances added to the {} intent".format(intent_name_greetings))

In [37]:
#add intent
add_greetings_intent(client, version_id, app_id)

#add utterances
add_utterances_greetings(client, version_id, app_id, nb_utterances_to_add=len(df_data_luis_greeting_intent_train))

Nb utterances_greetings to add = 70

Utterances added to the GreetingsIntent intent


## Ajout d'exemples pour l'intention : None
- Fonction **add_none_intent_additionnal_exmples** permettant d'ajouter des exemples supplémentaires
- Fonction **add_none_intent** permettant d'ajouter l'intention
- Fonction **add_utterances_none** permettant d'ajouter les utterances (les exemples)

In [38]:
df_data_luis_none_intent

,text,intent,or_city,dst_city,str_date,end_date,budget
0,"Have you ever read the book ""Vernon's Travels""?",none,,,,,
1,psssstttttt,none,,,,,
2,Vacay time woooohooooooo,none,,,,,


##### Remarque : étant donné que le nombre d'exemples pour l'intention None est très faible dans le jeu de données fourni, nous allons ajouter des exemples supplémentaires pour améliorer le modèle

### Exemples supplémentaires pour l'intention None

In [39]:
#Exemples éloignés du contexte
added_none_intent_1 = "The cat is in the kitchen"  
added_none_intent_2 = "Weather is cloudy"
added_none_intent_3 = "And then they were none"
added_none_intent_4 = "winter is coming"

#Exemples proches du contexte
added_none_intent_5 = "i'd like to rent a car"
added_none_intent_6 = "find an airport near me"
added_none_intent_7 = "I would like to book a restaurant"
added_none_intent_8 = "I need to book a hotel"

added_none_intents = [added_none_intent_1, added_none_intent_2, added_none_intent_3, added_none_intent_4, added_none_intent_5, added_none_intent_6, added_none_intent_7, added_none_intent_8]

### Fonction permettant l'ajout d'exemples supplémentaires pour l'intention None

In [40]:
def add_none_intent_additionnal_exmples(df_data_luis_none_intent, added_none_intents):
    for i, added_none_intent in enumerate(added_none_intents):
        df_data_luis_none_intent = df_data_luis_none_intent.append({'text' : added_none_intents[i], 
                                                                    'intent' : 'none',
                                                                    'or_city' : '',
                                                                    'dst_city' : '',
                                                                    'str_date' : '',
                                                                    'end_date' : '',
                                                                    'budget' : ''}, ignore_index=True)
        
    return df_data_luis_none_intent

In [41]:
df_data_luis_none_intent = add_none_intent_additionnal_exmples(df_data_luis_none_intent, added_none_intents)

In [42]:
df_data_luis_none_intent

,text,intent,or_city,dst_city,str_date,end_date,budget
0,"Have you ever read the book ""Vernon's Travels""?",none,,,,,
1,psssstttttt,none,,,,,
2,Vacay time woooohooooooo,none,,,,,
3,The cat is in the kitchen,none,,,,,
4,Weather is cloudy,none,,,,,
5,And then they were none,none,,,,,
6,winter is coming,none,,,,,
7,i'd like to rent a car,none,,,,,
8,find an airport near me,none,,,,,
9,I would like to book a restaurant,none,,,,,


In [43]:
intent_name_none = "NoneIntent"
def add_none_intent(client, version_id, app_id):
    #add intent
    intent_id_none = client.model.add_intent(app_id, version_id, intent_name_none)

In [44]:
def add_utterances_none(client, version_id, app_id, nb_utterances_to_add):

    utterances_none = []

    for i in range(min(nb_utterances_to_add, len(df_data_luis_none_intent), 100)):

        utterances_none.append({
            'text': df_data_luis_none_intent.iloc[i][0],
            'intent_name': intent_name_none,
        })
        
    print(f"Nb utterances_none to add = {len(utterances_none)}")
        
    utterances_result = client.examples.batch(app_id, version_id, utterances_none)
    
    print("\nUtterances added to the {} intent".format(intent_name_none))

In [45]:
#add intent
add_none_intent(client, version_id, app_id)

#add utterances
add_utterances_none(client, version_id, app_id, nb_utterances_to_add=len(df_data_luis_none_intent))

Nb utterances_none to add = 11

Utterances added to the NoneIntent intent


# Entrainement du modèle
- Fonction **train_model** permettant l'entrainement du modèle dans Azure

In [46]:
def train_model(client, version_id, app_id):
    # Training the model
    print("\nWe'll start training your app...")

    async_training = client.train.train_version(app_id, version_id)
    is_trained = async_training.status == "UpToDate"

    trained_status = ["UpToDate", "Success"]
    while not is_trained:
        time.sleep(1)
        status = client.train.get_status(app_id, version_id)
        is_trained = all(m.details.status in trained_status for m in status)

    print("Your app is trained. You can now go to the LUIS portal and test it!")

In [47]:
train_model(client, version_id, app_id)


We'll start training your app...
Your app is trained. You can now go to the LUIS portal and test it!


# Publication du modèle
- Fonction **publish_model** permettant la publication du modèle dans Azure

In [48]:
def publish_model(client, version_id, app_id):
    client.apps.update_settings(app_id, is_public=True)
    client.apps.publish(app_id, version_id, is_staging=False)

In [49]:
publish_model(client, version_id, app_id)

# Evaluation du modèle

## Creation du client de l'application (LUISRuntimeClient)

In [50]:
def create_luis_client_runtime():
    runtime_credentials = CognitiveServicesCredentials(prediction_key)
    client_runtime = LUISRuntimeClient(endpoint=prediction_endpoint, credentials=runtime_credentials)
    
    return client_runtime

In [51]:
client_runtime = create_luis_client_runtime()

## Evaluation sur le jeu de test

### Intention OrderTravel

In [52]:
df_data_luis_book_intent_test

,text,intent,or_city,dst_city,str_date,end_date,budget
0,Hey hey. Got 6 days off starting Sept 8. Flyin...,book,Cleveland,,Sept 8,,
1,"Hi yes, to Jerusalem please.",book,,Jerusalem,,,
2,You got any flights from New York to Naples de...,book,New York,Naples,Aug 27,Sep 15,
3,Hello. me and my grand child would like to go ...,book,,,,,
4,i need a trip under the radar from mannheim to...,book,mannheim,fukuoka,,august 30th,
...,...,...,...,...,...,...,...
562,Good afternoon. I'd like to book a business tr...,book,Mannheim,New York,,,
563,yes. i am going 2 bring my grand daughter with...,book,,,,,
564,This is a little short notice But I'm trying t...,book,San Diego,,Sept 8,17,
565,HI I NEED TO GO ON VACATION I LIVE IN WASHINGT...,book,,,,,


#### Fonction permettant de calculer le nombre d'intentions et d'entités correctement détectées par LUIS dans le jeu de test

In [53]:
def eval_order_travel_intent(df_data_luis_book_intent_test, test_size, app_id, client_runtime):

    correct_intents=0
    correct_entities=0

    list_strip = ['.', '!', '?', ',']
    
    for i in range(min(test_size, len(df_data_luis_book_intent_test))):

        text = df_data_luis_book_intent_test.iloc[i][0]
        predictionRequest = { "query" : text}

        predictionResponse = client_runtime.prediction.get_slot_prediction(app_id, "Production", predictionRequest)
        top_intent = predictionResponse.prediction.top_intent

        if (top_intent=="OrderTravelIntent"):
              correct_intents+=1

        departure_city = df_data_luis_book_intent_test.iloc[i][2]
        try:
            departure_city_predicted = predictionResponse.prediction.entities['DepartureCity'][0]
            for strip in list_strip :
                departure_city_predicted = departure_city_predicted.rstrip(strip)
            if departure_city_predicted == departure_city:
                correct_entities+=1
        except:
            if departure_city=='':
                correct_entities+=1

        arrival_city = df_data_luis_book_intent_test.iloc[i][3]
        try:
            arrival_city_predicted = predictionResponse.prediction.entities['ArrivalCity'][0]
            for strip in list_strip :
                arrival_city_predicted = arrival_city_predicted.rstrip(strip)
            if arrival_city_predicted == arrival_city:
                correct_entities+=1
        except:
            if arrival_city=='':
                correct_entities+=1

        departure_date = df_data_luis_book_intent_test.iloc[i][4]
        try:
            departure_date_predicted = predictionResponse.prediction.entities['DepartureDate'][0]
            for strip in list_strip :
                departure_date_predicted = departure_date_predicted.rstrip(strip)
            if departure_date_predicted == departure_date:
                correct_entities+=1
        except:
            if departure_date=='':
                correct_entities+=1

        arrival_date = df_data_luis_book_intent_test.iloc[i][5]
        try:
            arrival_date_predicted = predictionResponse.prediction.entities['ArrivalDate'][0]
            for strip in list_strip :
                arrival_date_predicted = arrival_date_predicted.rstrip(strip)
            if arrival_date_predicted == arrival_date:
                correct_entities+=1
        except:
            if arrival_date=='':
                correct_entities+=1

        price = df_data_luis_book_intent_test.iloc[i][6]
        try:
            price_predicted = predictionResponse.prediction.entities['Price'][0]
            for strip in list_strip :
                price_predicted = price_predicted.rstrip(strip)
            if price_predicted == price:
                correct_entities+=1
        except:
            if price=='':
                correct_entities+=1

    print(f"Pourcentage d'intention correctement détectées par LUIS = {correct_intents/test_size:.2f}")

    print(f"Pourcentage d'entitées correctement détectées par LUIS = {correct_entities/(test_size*5):.2f}")

In [54]:
test_size=100
eval_order_travel_intent(df_data_luis_book_intent_test, test_size, app_id, client_runtime)

Pourcentage d'intentions correctement détectées par LUIS = 0.99
Pourcentage d'entitées correctement détectées par LUIS = 0.84


### Intention Greeting

#### Fonction permettant de calculer le nombre d'intentions correctement détectées par LUIS dans le jeu de test

In [55]:
def eval_greeting_intent(df_data_luis_greeting_intent_test, test_size, app_id, client_runtime):

    correct_intents=0

    for i in range(min(test_size,len(df_data_luis_greeting_intent_test))):

        text = df_data_luis_greeting_intent_test.iloc[i][0]
        predictionRequest = { "query" : text}

        predictionResponse = client_runtime.prediction.get_slot_prediction(app_id, "Production", predictionRequest)
        top_intent = predictionResponse.prediction.top_intent

        if (top_intent=="GreetingsIntent"):
              correct_intents+=1

    print(f"Pourcentage d'intention correctement détectées par LUIS = {correct_intents/test_size:.2f}")   

In [56]:
test_size=len(df_data_luis_greeting_intent_test)
eval_greeting_intent(df_data_luis_greeting_intent_test, test_size, app_id, client_runtime)

Pourcentage d'intentions correctement détectées par LUIS = 0.85
